## Update and install packages

In [4]:
# Update and install display packages and stable baseline 3
!pip install -r ../requirements.txt
!pip install ipywidgets




[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Import Libraries

In [13]:
# Import rquired libraries and modules
import gymnasium as gym
from gymnasium import spaces
from gymnasium import Space
import stable_baselines3
from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import torch
from pathlib import Path
from typing import List
import random
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

import sys
import os

# Add the /app/src directory to the Python path
sys.path.append(os.path.abspath('../app/src'))


# Now try importing again
from spark.data import loader
from spark.data.models import Customer, Product, Category, Interaction, InteractionType
from spark import utils


In [14]:
# Check gym and stable baseline 3 versions
print(f"{gym.__version__=}")
print(f"{stable_baselines3.__version__=}")


gym.__version__='1.0.0'
stable_baselines3.__version__='2.4.0a11'


## Settings

### Tunning parameters

In [15]:
param_clip_range = 0.2
param_learning_rate = 0.0007
param_gamma=0.995
param_gae_lambda=0.95
param_n_steps=100
param_ent_coef = 0.01
param_total_timesteps = 3000000
save_interval = param_total_timesteps/10

### Other settings

In [16]:
# name of the environment to run
env_name = 'Spark'
env_prefix = 'spark'

# output directoies
base_dir = '.'
output_dir = os.path.join(base_dir, 'output')
env_dir =  os.path.join(output_dir, env_prefix)
logs_dir = os.path.join(env_dir, 'logs')
models_dir = os.path.join(env_dir, 'models')
videos_dir = os.path.join(env_dir, 'videos')

os.makedirs(logs_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)
os.makedirs(videos_dir, exist_ok=True)

# tensorboard name for algorithm logs
tb_log_name = 'A2C'
model_prefix = 'a2c'
model_name_final = f"{model_prefix}_model_final"

print(logs_dir)
print(models_dir)
print(videos_dir)

./output/spark/logs
./output/spark/models
./output/spark/videos


## Callbacks and Directory Setup

In [17]:
 # callback for saving model at regular intervals
class SaveOnIntervalCallback(BaseCallback):
    def __init__(self, save_interval: int, save_path: str, verbose=1):
        super().__init__(verbose)
        self.save_interval = save_interval
        self.save_path = save_path

    def _on_step(self) -> bool:
        # Save the model every 'save_interval' steps
        if self.num_timesteps % self.save_interval == 0:
            save_file = os.path.join(self.save_path, f'{model_prefix}_model_{self.num_timesteps}')
            self.model.save(save_file)
            if self.verbose > 0:
                print(f'Saving model to {save_file}.zip')
        return True

## Custom environment

In [18]:

"""
Custom gym environment for product recommendations where states represent customer interactions
actions are products;

Each customer interaction is a state. In the step function, transition of states will be the same 
customer with the new interaction for a product. The transition ends when the customer made a purchase or session ends.

The aim is to maximise rewards that can lead to a purchase.

It is better to have a customer interaction to represent a state rather than a time series of steps leading to a purchase. 
The latter method may have an incomplete where customer exits the application. Also, even if the customer did not purchase,
this information is still valuable for recommendations. Hence every state is a customer interaction.

An addition meta data for each customer will be stored to understand the context of the user. For example, if a product is 
purchase many times, this may factor into preference of the states.
---
Personalization: By incorporating the user ID, the model can tailor recommendations specifically to individual users, allowing 
it to learn unique user preferences and behaviors over time.

VS

Overfitting: If the model learns too much from the user ID directly, it might overfit to individual user patterns, potentially 
missing out on broader trends that could be useful for all users.

SOLUTION
Use Embeddings: Instead of directly using user IDs, consider using an embedding layer that transforms the user ID into a dense vector representation. This approach reduces dimensionality while capturing user-specific features.

Combine Features: Use user ID embeddings in conjunction with other features like user demographics, interaction history, and product attributes. This can create a more holistic view of user preferences.

Regularization: Implement techniques like dropout or weight regularization to mitigate overfitting when using user IDs or their embeddings.

Batch Normalization: Use batch normalization to stabilize learning, especially if the user ID leads to a wide range of outputs.
----
"""
class RecommendationEnv(gym.Env):
    def __init__(self, users:List[Customer], products:List[Product], top_k:int):
        super().__init__()
        
        self.users = users                  # list of users as states
        self.products = products            # products as actions, potential recommendations
        self.top_k = top_k                  # number of recommendations
        self.user_idx = 0                   # index of users list, not user_id
        self.current_step = 0               # step is also the interactions list index
        self.categories = loader.load_categories()
        
        self.action_space = spaces.MultiDiscrete([len(products)] * 10) 
        
        # number of customers as states
        # states are derived from customer profiles and interactioms
        # Users list will keep track of unique users
        # States include subset of features including product, interaction, ratings, and time in one-hot-encoding format
        # States exclude user_ids for policy network generalisation. But internal users list will be used as reference        
        self.observation_space = spaces.Dict({
            'pref_prod': spaces.Box(low=0, high=1, shape=(len(self.products),), dtype=np.float32),
            'pref_cat': spaces.Box(low=0, high=1, shape=(len(self.categories),), dtype=np.float32),
            'buys': spaces.Box(low=0, high=1000, shape=(len(self.products),), dtype=np.float32),
            'views': spaces.Box(low=0, high=1000, shape=(len(self.products),), dtype=np.float32),
            'likes': spaces.Box(low=0, high=1000, shape=(len(self.products),), dtype=np.float32),
            'ratings': spaces.Box(low=0, high=5, shape=(len(self.products),), dtype=np.uint8),
            'product': spaces.Box(low=0, high=1, shape=(len(self.products),), dtype=np.uint8),
            'interaction': spaces.Box(low=0, high=1, shape=(len(list(InteractionType)),), dtype=np.uint8),
            'rating': spaces.Discrete(6)
            }) 
            ## add more features like time, ignored recommendtions, engagement etc
        
    def reset(self, seed=None, options=None):
        # Call the parent class's reset method to handle seeding
        super().reset(seed=seed)
        
        self.user_idx = np.random.randint(len(self.users)) # may run throught users one by one
        user = self.users[self.user_idx]
        self.current_step = 0
        return self._get_observation(user), {}# get current user features as states

    def step(self, rec_products, interaction:Interaction=None):
        """ randomly interacting with product to mimick real user unpredictable behavious """
        self.current_step += 1
        
        # interaction passed in fro
        if interaction:    
            self.user_idx = interaction.customer_idx
        else:       
            # simulate selected recommended product and interaction
            user_id = self.user_idx
            interaction_time = datetime.now()
            selected_pid, interaction_type = self._simulate_interaction(rec_products) # generate random interaction
            random_rating = random.randint(0, 5)
            interaction = Interaction(-1, interaction_time, user_id, selected_pid, interaction_type, random_rating)
        
        reward = 0
        done = False
        
        # reward function if the recommended product is clicked
        if interaction.product_idx in rec_products:
            if interaction.type == InteractionType.NONE:
                reward = -1 # no interaction, customers not interested in recommendations
            elif interaction.type ==  InteractionType.VIEW:
                reward = 3
            elif interaction.type ==  InteractionType.LIKE:
                reward = 10
            elif interaction.type ==  InteractionType.BUY:
                reward = 50
            elif interaction.type ==  InteractionType.RATE:            
                reward = interaction.value - 2  # rating of 1 is negative
        
        done = interaction.type ==  InteractionType.SESSION_CLOSE
        
        interaction_info = {
                'interaction': {
                    'idx': -1,
                    'timestamp': interaction.timestamp.strftime("%Y-%m-%d %H:%M:%S"),
                    'customer_id': interaction.customer_idx,
                    'product_id': interaction.product_idx,
                    'interaction': interaction.type.value,
                    'rating': interaction.value
                }
            }
        
        user = self.users[self.user_idx]
        
        return self._update_observation(user, interaction), reward, done, False, interaction_info

    def _update_observation(self, user:Customer, interaction:Interaction):            
        if interaction.type == InteractionType.VIEW:
            user.views[interaction.product_idx] += 1
        elif interaction.type == InteractionType.LIKE:
            user.likes[interaction.product_idx] += 1
        elif interaction.type == InteractionType.BUY:
            user.buys[interaction.product_idx] += 1
        elif interaction.type == InteractionType.RATE:
            user.ratings[interaction.product_idx] = interaction.value
          
        # update observation based on new data  
        obs = {
                'pref_prod': self._get_product_preferences(user),
                'pref_cat': self._get_category_preferences(user), 
                'buys': user.buys,
                'views': user.views,
                'likes': user.likes,
                'ratings': user.ratings,
                'product': utils.one_hot_encode(interaction.product_idx, len(self.products)),
                'interaction': self._get_interaction_observation(interaction),
                'rating': interaction.value if interaction.type == InteractionType.RATE else 0 
            }
        
        return obs
    
    def update_observation(self, user:Customer, interaction:Interaction):
        return self._update_observation(user, interaction)

    def _get_observation(self, user:Customer): 
        
        obs = {
                'pref_prod': self._get_product_preferences(user),
                'pref_cat': self._get_category_preferences(user), 
                'buys': user.buys,
                'views': user.views,
                'likes': user.likes,
                'ratings': user.ratings,
                'product': np.zeros(len(self.products)),
                'interaction': np.zeros(len(list(InteractionType))),
                'rating': 0 
            }
        
        return obs
        
    def _get_interaction_observation(self, interaction:Interaction):
        idx = list(InteractionType).index(interaction.type)
        size = len(InteractionType)
        
        return utils.one_hot_encode(idx, size)
    
    # calculate preferences based on past interactions
    def _get_product_preferences(self, user:Customer):
        view_prefs = user.views / 20
        purchase_prefs = user.buys
        like_prefs = user.likes / 15

        rating_prefs = user.ratings.copy()
        rating_prefs[rating_prefs > 0] -= 2
        
        product_prefs = view_prefs + purchase_prefs + like_prefs+ rating_prefs
        
        product_prefs / 5 # reduce space
        
        return product_prefs    # calculate preferences based on past interactions
    
    def _get_category_preferences(self, user:Customer):
        prod_prefs = self._get_product_preferences(user)
        cat_prefs = np.zeros(len(self.categories), np.float32)
        
        for idx, prod_pref in enumerate(prod_prefs):
            if prod_pref > 0:
                product = self.products[idx]
                cat_idx = product.category.idx
                cat_prefs[cat_idx] += prod_pref # accumulation of fav products for this cat
                # print(f"added pf {prod_pref} to cat {cat_idx}")
                
        cat_prefs = cat_prefs / 5 # reduce space     
           
        return cat_prefs

    def _simulate_interaction(self, product_ids):        
        user = self.users[self.user_idx]
        product_list = []
        
        # simulate selection
        num_products = len(product_ids)
        prod_scores = np.zeros(num_products, np.uint8)
        product_prefs = self._get_product_preferences(user)
        category_prefs = self._get_category_preferences(user)
        product_probs = np.full((num_products,), 1.0 / num_products) # equal probs by default
        product_probs[-1] = 0.1 # lower ending epsidoe flag to encourage longer training
        
        for idx, pid in enumerate(product_ids):
            product_list.append(self.products[pid]) # get the product objects
            prod_scores[idx] = product_prefs[pid]
            
        # combining category prefs to calculate probabilities
        for idx, product in enumerate(product_list):
            cid = product.category.idx
            prod_scores[idx] = category_prefs[cid] 
    
        # Ensure the probabilities sum to 1 for a valid probability distribution
        if np.max(prod_scores) > 0: # the product is in the preferences
            product_probs = np.array(prod_scores) / sum(prod_scores)

        # Randomly select a product based on the defined probabilities
        selected_product_id = np.random.choice(product_ids, p=product_probs)
        
        # simulate interaction for the selected product
        inter_types = list(InteractionType)
        inter_scores = np.zeros(len(inter_types), np.uint8)
        inter_probs = np.full((len(inter_types),), 1.0 / len(inter_types)) # equal probs by default
        
        
        for idx, inter_type in enumerate(inter_types):
            if inter_type == InteractionType.VIEW:
                inter_scores[idx] = user.views[selected_product_id]
            if inter_type == InteractionType.LIKE:
                inter_scores[idx] = user.likes[selected_product_id]
            if inter_type == InteractionType.BUY:
                inter_scores[idx] = user.buys[selected_product_id]
            if inter_type == InteractionType.RATE:
                inter_scores[idx] = user.ratings[selected_product_id]
        
        if np.max(inter_scores) > 0:
            inter_scores[inter_scores == 0] = 1 # default score for interaction that are 0
            inter_probs = np.array(inter_scores) / sum(inter_scores)

        # Randomly select a product based on the defined probabilities
        selected_interaction_type = np.random.choice(inter_types, p=inter_probs)
        
        return selected_product_id, selected_interaction_type

    def seed(self, seed=None):
        """
        Set the seed for reproducibility.
        """
        self.np_random, seed = gym.utils.seeding.np_random(seed)
        random.seed(seed)
        np.random.seed(seed)
        return [seed]
    
    def render(self, mode='human'):
        if hasattr(self, 'last_action'):
            print(f"Recommended Product ID (Last Action): {self.last_action}")
        else:
            print("No product recommended yet.")

        # Optionally, print the reward received for the last action
        if hasattr(self, 'last_reward'):
            print(f"Reward for Last Action: {self.last_reward}")

        print("-----")

    def close(self):
        pass

## Initialise model with atari environemnt

In [19]:
products = loader.load_products()
customers = loader.load_customers(include_interactions=True)

In [20]:
len(products)

295

In [24]:
for customer in customers:
    customer.views = np.zeros(len(products), dtype=np.int8)
    customer.likes = np.zeros(len(products), dtype=np.int8)
    customer.buys = np.zeros(len(products), dtype=np.int8)
    customer.ratings = np.zeros(len(products), dtype=np.int8)
    for interaction in customer.interactions:
        i_type = interaction.type.value    
        product_idx = interaction.product_idx  
        if i_type == InteractionType.VIEW.value:
            customer.views[product_idx] += 1
        elif i_type == InteractionType.LIKE.value:
            customer.likes[product_idx] += 1
        elif i_type == InteractionType.BUY.value:
            customer.buys[product_idx] += 1
        elif i_type == InteractionType.RATE.value:
            customer.ratings[product_idx] = interaction.value

In [25]:
env = RecommendationEnv(customers, products, top_k=10)
env.seed(100)

[100]

## Initialize the PPO agent with specified parameters

In [26]:

model = A2C(
    env=env,
    policy='MultiInputPolicy',
    verbose=0,
    learning_rate=param_learning_rate, 
    n_steps=param_n_steps,
    ent_coef=param_ent_coef,
    gamma=param_gamma,
    gae_lambda=param_gae_lambda,
    tensorboard_log=logs_dir
)


## Model training

In [ ]:
# Define the interval at which models are saved during training
save_callback = SaveOnIntervalCallback(save_interval, models_dir)
# reward_callback = RewardLoggingCallback(log_file=f"{logs_dir}training_rewards_ppo.csv")
model.learn(total_timesteps=param_total_timesteps, progress_bar=True, callback=save_callback)

# Save the final model after training completes
final_model_path = os.path.join(models_dir, model_name_final)
model.save(final_model_path)

   1% ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30,905/3,000,000  [ 0:03:47 < 6:02:16 , 137 it/s ]
   1% ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30,905/3,000,000  [ 0:03:47 < 6:02:16 , 137 it/s ]

## Testing

In [ ]:
model =  A2C.load(os.path.join(models_dir,'a2c_model_final.zip'))

In [ ]:
# Construct observation data for customer 10
test_customer = customers[10] 
test_customer.views[200] = 3
# simulated laset selected product
test_product = products[100]

# it only uses the previous interaction to predict
previous_interaction = Interaction(idx = '0', 
                          timestamp = datetime.now(), 
                          customer_idx = test_customer.idx, 
                          product_idx = test_product.idx, 
                          type = InteractionType.VIEW,
                          value = 5,)

new_obs = env.update_observation(test_customer, previous_interaction)

In [ ]:
recommended_products, _states = model.predict(new_obs, deterministic=True)

In [ ]:
print(recommended_products)

In [ ]:
new_interaction = Interaction(idx = -1, 
                          timestamp = datetime.now(), 
                          customer_idx = test_customer.idx, 
                          product_idx = test_product.idx, 
                          type = InteractionType.LIKE,
                          value = 3,)

In [ ]:
obs, rewards, done, flag, interaction_info = env.step(recommended_products, new_interaction)

In [ ]:
recommended_products, _states = model.predict(obs, deterministic=True)

In [ ]:
print(recommended_products)

In [ ]:
interaction_info